In [95]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx
import ribasim

In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
# network name and results directory
network_name = "zutphen_tki_netwerk"

base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")

In [129]:
# create networkanalysis
network = RibasimLumpingNetwork(name=network_name)

In [130]:
# load areas units (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)[["geometry"]]
network.areas_gdf = areas_gdf.to_crs(28992)

In [131]:
# add data from simulation sets
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

Simulation set (winter): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc
Simulation set (zomer): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc


In [132]:
# get all network data from the schematisations
network.get_network_data()

network locations read: nodes/edges/confluences/bifurcations/weirs/pumps/laterals


In [138]:
# Define node_ids on which to split
# - define types to include
split_node_ids = network.get_node_ids_from_type(
    bifurcations=False,
    confluences=False,
    weirs=True,
    laterals=False
)

# - define by node code
split_node_ids_to_include = [1452, 2378]

# - define which of the node_ids should be excluded
split_node_ids_to_exclude = [314]

# - combine types of split_node_ids
split_node_ids = [node_id for node_id in split_node_ids + split_node_ids_to_include 
                  if node_id not in split_node_ids_to_exclude]

In [139]:
# create basins (gdf) based on nodes, edges, split_node_ids and areas
results = network.create_basins_based_on_split_node_ids(split_node_ids=split_node_ids)
# export it to geopackage
network.export_to_geopackage(output_dir=results_dir)

In [103]:
# get split_nodes in geodataframe
gpkg_path = Path(results_dir, network_name, 'ribasim_project.gpkg')
split_nodes_gdf = gpd.read_file(gpkg_path, layer='split_nodes')

In [104]:
# create basins (gdf) based on nodes, edges, split_node_ids and areas
results = network.create_basins_based_on_split_node_gdf(split_node_gdf=split_nodes_gdf)
# export it to geopackage
network.export_to_geopackage(output_dir=results_dir)